In [1]:
# Проверка возможности использования xgbooster в основном, либо предварительно (2017-06-30)

In [2]:
import sys,os,datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
from __future__ import print_function

In [3]:
print(np.__version__);
print(pd.__version__);

1.12.1
0.19.2


In [4]:
import  cv2 as cv
cv.__version__

'3.2.0'

In [5]:
sys.path.append('../Python')
from helper import formFH, paths_input

In [138]:
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential,save_model,load_model
from keras.layers import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
import keras.optimizers
from keras.callbacks import EarlyStopping

In [6]:
keras.__version__

'2.0.4'

In [7]:
trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG = paths_input()
if True :
    trDirI = trDirJPG
    teDirI = teDirJPG
    ttExt  = '.jpg'
else :
    trDirI = trDirTIF
    teDirI = teDirTIF
    ttExt  = '.tif'
trWork, teWork = '../Work/Train', '../Work/Test'
print (trDirI,teDirI, trWork, teWork)
print (trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG)

../Data/train-jpg ../Data/test-jpg-v2 ../Work/Train ../Work/Test
../Data/train_v2.csv ../Data/train-tif-v2 ../Data/train-jpg ../Data/test-tif-v2 ../Data/test-jpg-v2


In [12]:
xx=os.listdir(trDirI); xx.sort(); xx[:6]

['train_0.jpg',
 'train_1.jpg',
 'train_10.jpg',
 'train_100.jpg',
 'train_1000.jpg',
 'train_10000.jpg']

In [14]:
labels_df = pd.read_csv(trLabels)
labels_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [15]:
# Build list with unique labels
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [16]:
# Add features (0 or 1) for every label as dataframe field 
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
#labels_df.head()

In [17]:
weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
land_labels = ['primary', 'agriculture', 'water', 'cultivation', 'habitation' ]
rare_labels = [l for l in label_list if labels_df[label_list].sum()[l] < 2000]
#rare_labels              

In [18]:
labels = label_list; #weather_labels;
nameList =labels_df[labels_df[labels].sum(axis=1)>0].image_name.tolist(); len(nameList)
labelList=labels_df[labels_df[labels].sum(axis=1)>0][labels].as_matrix();
labelList[:6,:]
#labels_df.head()

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [30]:
def formImExt (nf, resize=(32,32), printOK=False) :
    nx = None
    try : 
        ni = cv.imread(nf,-1); 
        if (ni is not None) :
            if not ((ni.shape[2]==3) or (ni.shape[2]==4)) and printOK : print('----- error ---- shape:',ni.shape,nf)
            if (ni.shape[2]==3) :
                nx = cv.resize(ni,resize)
            if (ni.shape[2]==4) :
                #r,g,b,n = ni[:,:,2],ni[:,:,1],ni[:,:,0],ni[:,:,3]
                r,g,b,n = cv.resize(ni[:,:,2],resize),cv.resize(ni[:,:,1],resize),cv.resize(ni[:,:,0],resize),cv.resize(ni[:,:,3],resize)
                dv,dw   = np.divide((r-n),(r+n+0.01)), np.divide((g-n),(g+n+0.01))
                nx      = np.array([r,g,b,n,dv,dw]).T; 
    except BaseException as e :
        print(nf,e); nx = None;
    
    if nx is None and printOK : 
        print('------ None:',nf); nx = None
        
    return(nx)

def formImHist (nf, size, printOK=False) :
    
    def hist1 (nf1,bins) :
        h,_ = np.histogram(nf1.ravel(),bins=bins)
        return h.tolist()
    
    def histN (nf,bins) :
        h = []
        for i in range(0,nf.shape[2]) :
            h = h + hist1(nf[:,:,i],bins=bins);
        return h
             
    nx = None
    try : 
        ni = cv.imread(nf,-1);
        if (ni is not None) :
            
            if printOK : print(nf,ni.shape)

            if not ((ni.shape[2]==3) or (ni.shape[2]==4)) and printOK : 
                print('----- error ---- shape:',ni.shape,nf);
            if  ni.shape[2]==3 :
                bins = [x for x in range(0,255+int(round(256.0/size)),int(round(256.0/size)))]
                nx = histN(ni,bins)
            if  ni.shape[2]==4 : 
                bins = [x for x in range(0,65535+int(round(65535.0/size)),int(round(65535.0/size)))]
                nx = histN(ni,bins)
            
    except BaseException as e :
        print(nf,e); nx = None;
    
    if nx is None and printOK : 
        print('------ None:',nf); nx = None
        
    return(nx)

In [31]:
os.path.exists(nf)
ni = cv.imread(nf,-1);
ni.shape

(256, 256, 3)

In [81]:
trX, trY, i, size = [],[], 0, len(nameList)
print(datetime.datetime.now(),len(nameList),size)
for nn in nameList[0:size] :
    nf = os.path.join(trDirI,nn+".jpg");
    nx = formImHist(nf,50,printOK=False)
    if (nx is not None) :
        #rr=np.save(os.path.join(trWork,nn+".npy"),nx);
        #trX.append(nn+".npy")
        trX.append(nx)
        trY.append(True)
    else : 
        trY.append(False)
    i += 1
    if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)
    #print(nn.shape)

print(datetime.datetime.now())
trX = np.array(trX);
trY = labelList[trY];
print(len(nameList),trX.shape,trY.shape)
#trXY=pd.DataFrame(trY); trXY['name']=trX; trXY.head()
#trXY.to_pickle(os.path.join(trWork,"listFiles.pkl"))

2017-07-01 02:24:35.495745 40479 40479
2017-07-01 02:25:06.489903 	 5000 	 train_4999
2017-07-01 02:25:38.274466 	 10000 	 train_9999
2017-07-01 02:26:10.440038 	 15000 	 train_14999
2017-07-01 02:26:42.506014 	 20000 	 train_19999
2017-07-01 02:27:14.630410 	 25000 	 train_24999
2017-07-01 02:27:46.054680 	 30000 	 train_29999
2017-07-01 02:28:18.267508 	 35000 	 train_34999
2017-07-01 02:28:49.803443 	 40000 	 train_39999
2017-07-01 02:28:52.914214
40479 (40479, 153) (40479, 17)


In [82]:
trX.shape,trY.shape

((40479, 153), (40479, 17))

In [83]:
np.save('../Work/train-data-XX.npy',trX)
np.save('../Work/train-data-YY.npy',trY)

In [84]:
del(trX); del(trY);

In [85]:
trX = np.load('../Work/train-data-XX.npy')
trY = np.load('../Work/train-data-YY.npy')
print(trX.shape,trY.shape)

(40479, 153) (40479, 17)


In [86]:
import xgboost as xgb

In [87]:
# Строим основные массивы
dtrain = xgb.DMatrix(trX,label=trY[:,1])

In [160]:
param = {
         'objective':'reg:logistic', 
         #'num_class':17,
         #'eta':0.05, 
         #'max_depth':9, 'subsample':0.8, 'colsample_bytree':0.8, 
         #'metrics':'rmse'
         'eval_metric':['auc','rmse']
        }
param['silent']=True

results, bstList = [], []
for i in range(0,trY.shape[1]) :
    print ('Working: {}'.format(i))
    dtrain.set_label(trY[:,i])
    history = xgb.cv(param, dtrain, metrics=['auc'], verbose_eval=30, num_boost_round=1000, nfold=3, early_stopping_rounds=20) #,stratified=True,maximize=True)
    results.append(history)
    rvalidCV = {}
    watchlistCV  = [(dtrain,'train')]
    bst = xgb.train(param, dtrain, history['test-auc-mean'].argmax()+1, watchlistCV) 
    bstList.append(bst)
    

Working: 0
[0]	train-auc:0.946892+0.00269225	test-auc:0.934311+0.00428495
[30]	train-auc:0.987084+0.000259243	test-auc:0.958988+0.00215864
[60]	train-auc:0.995101+0.00036585	test-auc:0.9591+0.00169753
[0]	train-auc:0.944702	train-rmse:0.377035
[1]	train-auc:0.95182	train-rmse:0.300587
[2]	train-auc:0.959234	train-rmse:0.250904
[3]	train-auc:0.961286	train-rmse:0.219585
[4]	train-auc:0.964187	train-rmse:0.201259
[5]	train-auc:0.965752	train-rmse:0.189527
[6]	train-auc:0.966423	train-rmse:0.182661
[7]	train-auc:0.967827	train-rmse:0.178723
[8]	train-auc:0.969109	train-rmse:0.17545
[9]	train-auc:0.970818	train-rmse:0.173626
[10]	train-auc:0.97328	train-rmse:0.172283
[11]	train-auc:0.973739	train-rmse:0.171344
[12]	train-auc:0.974678	train-rmse:0.170305
[13]	train-auc:0.975333	train-rmse:0.169601
[14]	train-auc:0.975721	train-rmse:0.169228
[15]	train-auc:0.976313	train-rmse:0.168322
[16]	train-auc:0.977328	train-rmse:0.16655
[17]	train-auc:0.978025	train-rmse:0.166076
[18]	train-auc:0.9783

In [112]:
#results[0]

In [113]:
#[(x['test-rmse-mean'].min(),x['test-rmse-mean'].argmin()) for x in results]

In [114]:
#[(x['test-rmse-mean'].min(),x['test-rmse-mean'].argmin()) for x in results]

In [161]:
[(x['test-auc-mean'].max(),x['test-auc-mean'].argmax()) for x in results]

[(0.95933766666666676, 51),
 (0.97132033333333334, 70),
 (0.92636800000000008, 84),
 (0.97527066666666673, 52),
 (0.82370966666666667, 91),
 (0.93564866666666668, 41),
 (0.91020533333333331, 58),
 (0.84959400000000007, 84),
 (0.84758433333333338, 38),
 (0.98782933333333334, 197),
 (0.9787530000000001, 214),
 (0.95955733333333326, 49),
 (0.86465266666666662, 54),
 (0.98490033333333338, 55),
 (0.88952866666666663, 32),
 (0.87565166666666672, 27),
 (0.81941499999999989, 40)]

In [255]:
# Forming output dataset for predicting --> trOX, trOY
del(trX)
del(trY)

In [106]:
nameAsk = os.listdir(teDirI); print(len(nameAsk))
trOX, trOY, i, size = [], [], 0, len(nameAsk)
print(datetime.datetime.now(),len(nameList),size)
for nn in nameAsk[0:size] :
    nf = os.path.join(teDirI,nn);
    nx = formImHist(nf,50) #,printOK=True)
    if (nx is not None) :
        trOX.append(nx)
        trOY.append(nn)
    i += 1
    if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)

print(datetime.datetime.now())

#assert (size!=len(trOY)), "Wrong files {} != {}".format(size,len(trOY))

61191
2017-07-01 03:00:11.909981 40479 61191
2017-07-01 03:00:43.603291 	 5000 	 test_14523.jpg
2017-07-01 03:01:16.074007 	 10000 	 test_19029.jpg
2017-07-01 03:01:47.652768 	 15000 	 test_23524.jpg
2017-07-01 03:02:18.740980 	 20000 	 test_28015.jpg
2017-07-01 03:02:49.361504 	 25000 	 test_32520.jpg
2017-07-01 03:03:19.945605 	 30000 	 test_37026.jpg
2017-07-01 03:03:50.749942 	 35000 	 test_4908.jpg
2017-07-01 03:04:22.473341 	 40000 	 test_9402.jpg
2017-07-01 03:04:53.815660 	 45000 	 file_13913.jpg
2017-07-01 03:05:24.655782 	 50000 	 file_18419.jpg
2017-07-01 03:06:06.729415 	 55000 	 file_4564.jpg
2017-07-01 03:06:41.395364 	 60000 	 file_892.jpg
2017-07-01 03:06:49.139797


In [107]:
trOX = np.array(trOX);
trOY = np.array([os.path.splitext(x)[0] for x in trOY]);
print(len(nameAsk),trOX.shape,trOY.shape)

61191 (61191, 153) (61191,)


In [135]:
#trOX[20],trOY[20]

In [109]:
# Saving & Loading

In [110]:
np.save('../Work/test-data-XX.npy',trOX)
np.save('../Work/test-data-YY.npy',trOY)

In [162]:
trOX = np.load('../Work/test-data-XX.npy')
trOY = np.load('../Work/test-data-YY.npy')
print(trOX.shape,trOY.shape)

(61191, 153) (61191,)


In [164]:
# Формирование результата
dtest  = xgb.DMatrix(trOX)

In [165]:
trP = []
for bstI in bstList :
    trP1 = bstI.predict(dtest);
    trP.append(trP1)
trP = np.array(trP).T
type(trP),trP.shape    

(numpy.ndarray, (61191, 17))

In [167]:
np.save('../Work/test-data-RR.npy',trP)

In [168]:
trP = np.load('../Work/test-data-RR.npy')
trP.shape

(61191, 17)

In [174]:
wr = [labels.index(i) for i in weather_labels];
trM=np.array([0.5]*17);
trM[np.array(wr)] = 2.0
print(wr,'\n',labels,'\n',trM.tolist())

[3, 10, 0, 9] 
 ['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down'] 
 [2.0, 0.5, 0.5, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 2.0, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


In [179]:
#trP = model.predict(trX, batch_size=512); trP=K.get_value(trP)
res = []

for i in range(trP.shape[0]) :
    trA1 = [weather_labels[trP[i,wr].argmax()]]
    trA2 = [labels[ii] for ii in range(len(labels)) if (trP[i,ii]>trM[ii])]
    trPP = trA1 + trA2;
    pp   = ' '.join(trPP)
    ##if (pp=="") : print(trY[i])
    res.append([trOY[i],pp])

res.sort(cmp=lambda x,y: cmp(int(x[0].partition('_')[2]),int(y[0].partition('_')[2])) if (x[0].partition('_')[0]==y[0].partition('_')[0]) else cmp(y[0].partition('_')[0],x[0].partition('_')[0]))
#print(res[4:8],'\n',res[-4:])
print(res[0],'\n',res[-1],'\n',trM,trP[0])

['test_0', 'clear primary'] 
 ['file_20521', 'clear primary'] 
 [ 2.   0.5  0.5  2.   0.5  0.5  0.5  0.5  0.5  2.   2.   0.5  0.5  0.5  0.5
  0.5  0.5] [  8.63761827e-02   1.52730331e-01   7.28085488e-02   4.67409104e-01
   4.57194686e-01   3.28843738e-03   6.79126661e-03   1.19121477e-03
   3.18192360e-05   7.46353567e-01   1.07324135e-03   1.07767019e-05
   4.12525143e-03   1.96425954e-05   1.57556555e-04   4.30482993e-04
   4.97959591e-05]


In [180]:
print(labels)
print(trM.tolist())
np.round(trP[4:11,:])

['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down']
[2.0, 0.5, 0.5, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 2.0, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


array([[ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.]], dtype=float32)

In [181]:
rrr=pd.DataFrame(res,columns=['image_name','tags']); rrr.head(); 
suffixDT = (datetime.datetime.now()).strftime('%Y-%m-%d-%H-%M-%S'); print(suffixDT)
rrr.to_csv('../Result/vss'+suffixDT+'.csv',index=False);

2017-07-01-04-21-26
